In [ ]:
%matplotlib inline


# Maxwell 3D: Magnetostatic Mutual Inductance Analysis


In [1]:
from pyaedt import Maxwell3d
from pyaedt import generate_unique_project_name
import os

## Launch AEDT
Launch AEDT 2022 R2 in graphical mode.



In [3]:
m3d = Maxwell3d(projectname=generate_unique_project_name(), specified_version="2022.2", new_desktop_session=True, non_graphical=False)

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\v222\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2022.2 Started with process ID 27580.
pyaedt INFO: pyaedt v0.6.28
pyaedt INFO: Python version 3.10.5 | packaged by conda-forge | (main, Jun 14 2022, 06:57:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: Project Project_QJQ has been created.
pyaedt INFO: No design is present. Inserting a new design.
pyaedt INFO: Added design 'Maxwell 3D_CFY' of type Maxwell 3D.
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded
pyaedt INFO: aedt file load time 0.0020012855529785156


## Set up Maxwell solution
Set up the Maxwell solution to DC.



In [4]:
m3d.solution_type = m3d.SOLUTIONS.Maxwell3d.Magnetostatic
m3d.modeler.model_units = 'um'

In [79]:
m3d.modeler.delete() # delete all objects to start fresh
for setup in m3d.setup_names:
    m3d.delete_setup(setup)
length = 50
width = 5
thickness = 5
material = 'copper'
air = m3d.modeler.create_box([-50, -50, 0], [100, 100, length], name="Air", matname='air')
wire1 = m3d.modeler.create_box([-0.75*width - width/2, 0, 0], [width, thickness, length], name="Wire1", matname=material)
wire2 = m3d.modeler.create_box([1.5/2*width - width/2, 0, 0], [width, thickness, length], name="Wire2", matname=material)

# m3d.modeler.unite([wire1,wire2])

c1top = m3d.assign_current(wire1.bottom_face_z, 5, name='c1top')
c1bottom = m3d.assign_current(wire1.top_face_z, 5, swap_direction=True, name='c1bottom')

c2top = m3d.assign_current(wire2.bottom_face_z, 5, name='c2top')
c2bottom = m3d.assign_current(wire2.top_face_z, 5, swap_direction=True, name='c2bottom')
# wire1.rotate(cs_axis='z', angle=45)

m3d.assign_matrix(['c1top', 'c2top'], matrix_name='current_matrix')
solSetup = m3d.create_setup(setupname='Sol1', props={'MinimumPasses': 3})
solSetup.props['MinimumPasses'] = 3
# solSetup.available_properties

pyaedt INFO: Deleted 3 Objects: Air,Wire1,Wire2.


In [101]:
m3d.create_output_variable('kappa','current_matrix.CplCoef(c2top,c1top)')
m3d.create_output_variable('L1','current_matrix.L(c1top,c1top)')
m3d.create_output_variable('L2','current_matrix.L(c2top,c2top)')
m3d.create_output_variable('M','current_matrix.L(c1top,c2top)')

True

In [108]:
kappa = m3d.get_output_variable('kappa')
L1 = m3d.get_output_variable('L1')
L2 = m3d.get_output_variable('L2')
M = m3d.get_output_variable('M')
print('k=',kappa)
print('L1=',L1,', L2=',L2)
print('M=',M)
h = 6.62607015e-34  # m2 kg / s
e = 1.60217663e-19 # C
fluxquant = h / (2 * e)
I_needed = fluxquant / M # A
print('current that achieves one flux quantum: ', I_needed * 1000, 'mA')

jcNb = 2e5 # A/cm^2
Nb_thickness = 100e-9 * 100 # cm
track_width = I_needed / jcNb / Nb_thickness #cm
print('Required min trackwidth: ', track_width*1e4, 'um')

k= 0.606471766866992
L1= 3.2418427969896603e-11 , L2= 3.25098720413542e-11
M= 1.96885708870901e-11
current that achieves one flux quantum:  0.10502711778742563 mA
Required min trackwidth:  0.5251355889371282 um


In [ ]:
m3d.save_project()
m3d.release_desktop(close_projects=True, close_desktop=True)